# Amber Prep Server - Integration Test

This notebook tests the `amber_prep_server.py` MCP server tools for converting
Boltz-2 mmCIF output to MD simulation input files.

## Workflow Overview

1. Parse Boltz-2 mmCIF complex → Separate protein/ligand
2. Prepare protein with pdb4amber
3. Add hydrogens to ligand (OpenBabel)
4. Estimate net charge (RDKit)
5. Run antechamber (GAFF2 + AM1-BCC)
6. Validate frcmod
7. Build system with tleap


In [ ]:
# Setup path
import sys
sys.path.insert(0, '..')


In [ ]:
# Test imports
from pathlib import Path
import json

# Verify key dependencies
try:
    import gemmi
    print(f"✓ gemmi available")
except ImportError:
    print("✗ gemmi not installed: pip install gemmi")

try:
    from rdkit import Chem
    print(f"✓ RDKit available")
except ImportError:
    print("✗ RDKit not installed (install via conda)")

try:
    from common.base import BaseToolWrapper
    print(f"✓ common.base available")
except ImportError:
    print("✗ common.base not found")


In [ ]:
# Check AmberTools availability
from common.base import BaseToolWrapper

tools = [
    ("antechamber", "mcp-md"),
    ("parmchk2", "mcp-md"),
    ("pdb4amber", "mcp-md"),
    ("tleap", "mcp-md"),
    ("obabel", "mcp-md")
]

for tool_name, env in tools:
    wrapper = BaseToolWrapper(tool_name, conda_env=env)
    status = "✓" if wrapper.is_available() else "✗"
    print(f"{status} {tool_name}: {wrapper.executable or 'NOT FOUND'}")


## Test 1: Charge Estimation

Test the `estimate_net_charge` function with sample molecules.


In [ ]:
# Reload module to get latest changes
import importlib
import servers.amber_prep_server
importlib.reload(servers.amber_prep_server)

# Import the server functions (after reload)
from servers.amber_prep_server import (
    estimate_net_charge,
    _estimate_charge_rdkit,
    _estimate_physiological_charge
)
from rdkit import Chem
from rdkit.Chem import AllChem

# Debug: Test SMARTS patterns directly
print("=== SMARTS Pattern Debug ===")
test_smiles = "CC(=O)O"  # Acetic acid
mol = Chem.MolFromSmiles(test_smiles)
mol_h = Chem.AddHs(mol)

patterns_to_test = [
    ("[CX3](=O)[OX2H1]", "Standard COOH"),
    ("[CX3](=O)[OH]", "Simplified"),
    ("[C](=O)[O;H1]", "General with H1"),
    ("[C](=O)O", "Most basic"),
    ("C(=O)O", "Minimal"),
]

for smarts, desc in patterns_to_test:
    pattern = Chem.MolFromSmarts(smarts)
    if pattern:
        matches_no_h = mol.GetSubstructMatches(pattern)
        matches_with_h = mol_h.GetSubstructMatches(pattern)
        print(f"  {desc}: without H={len(matches_no_h)}, with H={len(matches_with_h)}")
    else:
        print(f"  {desc}: INVALID SMARTS")

print()

# Test molecules with known charges at pH 7.4
test_cases = [
    ("CCO", 0, "Ethanol - neutral"),
    ("CC(=O)O", -1, "Acetic acid - deprotonated at pH 7.4"),
    ("CCN", +1, "Ethylamine - protonated at pH 7.4"),
    ("c1ccc(cc1)C(=O)O", -1, "Benzoic acid"),
    ("NCCCC[C@H](N)C(=O)O", +1, "Lysine - free amino acid (2×NH3+, COO-)"),
    ("CC(C)C[C@H](N)C(=O)O", 0, "Leucine - (NH3+, COO-)"),
]

print("=== Charge Estimation Tests ===")
for smiles, expected, name in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    
    # Direct pattern test for debugging
    carboxylic_pattern = Chem.MolFromSmarts("C(=O)O")
    carboxylic_matches = mol.GetSubstructMatches(carboxylic_pattern) if carboxylic_pattern else []
    
    charge_info = _estimate_charge_rdkit(mol)
    estimated = _estimate_physiological_charge(charge_info)
    
    status = "✓" if estimated == expected else "✗"
    print(f"{status} {name}: formal={charge_info['formal_charge']}, estimated={estimated}, expected={expected}")
    print(f"   Direct COOH matches: {len(carboxylic_matches)}")
    if charge_info['ionizable_groups']:
        for group in charge_info['ionizable_groups']:
            print(f"   - {group['type']}: {group['count']} × {group['typical_charge']}")


## Test 2: frcmod Validation

Test the `_parse_frcmod_warnings` function.


In [ ]:
from servers.amber_prep_server import _parse_frcmod_warnings
from pathlib import Path
import tempfile

# Create a test frcmod with warnings
test_frcmod_content = """Remark line goes here
MASS

BOND
c3-os  301.5   1.4340  ATTN, need revision
ca-os  372.0   1.3730

ANGLE
c3-c3-os   67.8    108.42
c3-os-c1   60.0    109.50   ATTN, need revision

DIHE
X -c3-os-X    3    1.150         0.0             3.0

IMPROPER

NONBON

"""

with tempfile.NamedTemporaryFile(mode='w', suffix='.frcmod', delete=False) as f:
    f.write(test_frcmod_content)
    test_frcmod_path = Path(f.name)

validation = _parse_frcmod_warnings(test_frcmod_path)
print(f"Valid: {validation['valid']}")
print(f"ATTN count: {validation['attn_count']}")
print(f"Warnings:")
for w in validation['warnings']:
    print(f"  - {w}")
print(f"Missing params: {validation['missing_params']}")

# Cleanup
test_frcmod_path.unlink()


## Test 3: sqm Output Parsing

Test the `_parse_sqm_output` function with simulated error outputs.


In [ ]:
from servers.amber_prep_server import _parse_sqm_output
import tempfile

# Test case 1: Odd electron error
sqm_odd_electrons = """            --------------------------------------------------------
                         AMBER SQM VERSION 19
 
                                   By
              Ross C. Walker, Michael F. Crowley, Scott Brozell,
                         Tim Giese, Andreas W. Goetz,
                        Tai-Sung Lee and David A. Case
 
            --------------------------------------------------------
 

QMMM: INFO: The number of electrons is odd (51)
QMMM: Fatal Error!
QMMM: Cannot properly run "sqm"
"""

with tempfile.NamedTemporaryFile(mode='w', suffix='.out', delete=False) as f:
    f.write(sqm_odd_electrons)
    test_sqm_path = Path(f.name)

diag = _parse_sqm_output(test_sqm_path)
print("Test 1: Odd electrons error")
print(f"  Success: {diag['success']}")
print(f"  Errors: {diag['errors']}")
print(f"  Recommendations: {diag['recommendations']}")
test_sqm_path.unlink()

# Test case 2: SCF convergence failure
sqm_scf_fail = """            --------------------------------------------------------
                         AMBER SQM VERSION 19
 
QMMM: No convergence in SCF after 1000 steps
QMMM: Unable to achieve self consistency
"""

with tempfile.NamedTemporaryFile(mode='w', suffix='.out', delete=False) as f:
    f.write(sqm_scf_fail)
    test_sqm_path = Path(f.name)

diag = _parse_sqm_output(test_sqm_path)
print("\nTest 2: SCF convergence failure")
print(f"  Success: {diag['success']}")
print(f"  SCF converged: {diag['scf_converged']}")
print(f"  Errors: {diag['errors']}")
print(f"  Recommendations: {diag['recommendations']}")
test_sqm_path.unlink()


## Test 4: Server Import Check

Verify all MCP tools are importable.


In [ ]:
# Import all tools from the server
from servers.amber_prep_server import (
    parse_boltz2_complex,
    estimate_net_charge,
    prepare_ligand_hydrogens,
    prepare_protein_for_amber,
    run_antechamber_robust,
    validate_frcmod,
    build_complex_system,
    boltz2_to_amber_complete
)

tools = [
    parse_boltz2_complex,
    estimate_net_charge,
    prepare_ligand_hydrogens,
    prepare_protein_for_amber,
    run_antechamber_robust,
    validate_frcmod,
    build_complex_system,
    boltz2_to_amber_complete
]

print("MCP Tools available:")
for tool in tools:
    # Handle both FunctionTool objects and regular functions
    tool_name = getattr(tool, 'name', None) or getattr(tool, '__name__', str(tool))
    tool_doc = getattr(tool, 'description', None) or getattr(tool, '__doc__', '')
    
    print(f"  ✓ {tool_name}")
    if tool_doc:
        first_line = tool_doc.strip().split('\n')[0]
        print(f"      {first_line}")


## Summary

The `amber_prep_server.py` provides the following MCP tools:

| Tool | Description |
|------|-------------|
| `parse_boltz2_complex` | Parse mmCIF, separate protein/ligand |
| `estimate_net_charge` | Auto-estimate ligand charge at target pH |
| `prepare_ligand_hydrogens` | Add hydrogens with OpenBabel |
| `prepare_protein_for_amber` | pdb4amber preparation |
| `run_antechamber_robust` | GAFF2 + AM1-BCC with error handling |
| `validate_frcmod` | Check for missing parameters |
| `build_complex_system` | tleap system building |
| `boltz2_to_amber_complete` | Complete end-to-end workflow |


## Test 5: Real Boltz-2 Output Processing

Process an actual Boltz-2 mmCIF output file.


In [ ]:
# Test with actual Boltz-2 output
# Note: MCP tools are FunctionTool objects, need to access underlying function
import importlib
import servers.amber_prep_server as amber_module
importlib.reload(amber_module)

from pathlib import Path

# Get the underlying function from FunctionTool or use directly
def get_callable(tool):
    """Extract callable from FunctionTool or return as-is if already callable"""
    if callable(tool):
        return tool
    # Try common attributes for wrapped functions
    for attr in ['fn', 'func', '_func', 'function', '_fn']:
        if hasattr(tool, attr):
            fn = getattr(tool, attr)
            if callable(fn):
                return fn
    raise TypeError(f"Cannot extract callable from {type(tool)}")

# Access the parse function
parse_boltz2_complex = get_callable(amber_module.parse_boltz2_complex)

# Path to real Boltz-2 output
boltz2_cif = Path("boltz_results_ligand/predictions/ligand/ligand_model_0.cif")

if boltz2_cif.exists():
    print(f"Processing Boltz-2 output: {boltz2_cif}")
    print("=" * 60)
    
    try:
        result = parse_boltz2_complex(str(boltz2_cif))
        
        print(f"\n✓ Job ID: {result['job_id']}")
        print(f"✓ Output directory: {result['output_dir']}")
        print(f"✓ Protein chains: {result['num_protein_chains']}")
        print(f"✓ Ligands found: {result['num_ligands']}")
        
        print(f"\n--- Chain Information ---")
        for chain in result['chains']:
            print(f"  {chain['name']}: {chain['type']}")
        
        print(f"\n--- Output Files ---")
        print(f"  Protein PDB: {result['protein_pdb']}")
        for lig in result['ligand_files']:
            print(f"  Ligand: {lig}")
        
        if result['ligand_smiles']:
            print(f"\n--- Extracted SMILES ---")
            for lig_info in result['ligand_smiles']:
                print(f"  {lig_info['residue']}: {lig_info['smiles'][:50]}...")
                
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"✗ Boltz-2 output file not found: {boltz2_cif}")


In [ ]:
# Test CORRECT workflow: Add hydrogens FIRST, then estimate charge
# For complex molecules (e.g., SAH), allow manual charge specification
# Use get_callable helper from previous cell
estimate_net_charge = get_callable(amber_module.estimate_net_charge)
prepare_ligand_hydrogens = get_callable(amber_module.prepare_ligand_hydrogens)

# Known charges for complex molecules (can be specified by user or LLM)
# SAH (S-adenosyl-L-homocysteine): -1 at pH 7.4 (COO- group)
KNOWN_CHARGES = {
    "SAH": -1,  # S-adenosyl-L-homocysteine
    # Add other known molecules here as needed
}

# Check if we have ligand files from previous cell
if 'result' in dir() and result.get('ligand_files'):
    print("Testing correct workflow: Hydrogens → Charge Estimation")
    print("(Complex molecules use manually specified charges)")
    print("=" * 60)
    
    job_dir = Path(result['output_dir'])
    
    # Store results for later use
    ligand_results = []
    
    for lig_file in result['ligand_files']:
        lig_path = Path(lig_file)
        if lig_path.exists():
            # Extract residue name from filename (e.g., ligand_SAH_chainC.pdb -> SAH)
            res_name = lig_path.stem.split('_')[1] if '_' in lig_path.stem else "UNK"
            
            print(f"\n{'='*60}")
            print(f"Ligand: {lig_path.name} (residue: {res_name})")
            print(f"{'='*60}")
            
            lig_result = {
                "file": str(lig_path),
                "residue": res_name,
                "charge": None,
                "charge_source": None,
                "h_file": None
            }
            
            try:
                # Step 1: Add hydrogens with pKa-aware protonation
                print("\n[Step 1] Adding hydrogens (OpenBabel, pH=7.4)...")
                h_result = prepare_ligand_hydrogens(
                    str(lig_path), 
                    output_dir=str(job_dir),
                    ph=7.4,
                    output_format="mol2"
                )
                print(f"  ✓ Output: {Path(h_result['output_file']).name}")
                print(f"  ✓ Hydrogens added: {h_result['num_hydrogens_added']}")
                print(f"  ✓ Total atoms: {h_result['num_atoms']}")
                lig_result["h_file"] = h_result['output_file']
                
                # Step 2: Estimate charge or use known value
                print("\n[Step 2] Determining net charge...")
                
                # Check if we have a known charge for this molecule
                if res_name in KNOWN_CHARGES:
                    charge = KNOWN_CHARGES[res_name]
                    print(f"  → Using KNOWN charge for {res_name}: {charge}")
                    lig_result["charge"] = charge
                    lig_result["charge_source"] = "manual"
                else:
                    # Try automatic estimation
                    try:
                        charge_result = estimate_net_charge(h_result['output_file'], ph=7.4)
                        print(f"  Formal charge: {charge_result['formal_charge']}")
                        print(f"  Estimated at pH 7.4: {charge_result['estimated_charge_at_ph']}")
                        print(f"  Confidence: {charge_result['confidence']}")
                        print(f"  Molecular formula: {charge_result['molecular_formula']}")
                        print(f"  Atoms: {charge_result['num_atoms']} ({charge_result['num_heavy_atoms']} heavy)")
                        
                        if charge_result['ionizable_groups']:
                            print(f"  Ionizable groups:")
                            for group in charge_result['ionizable_groups']:
                                print(f"    - {group['type']}: {group['count']} (charge: {group['typical_charge']})")
                        
                        lig_result["charge"] = charge_result['estimated_charge_at_ph']
                        lig_result["charge_source"] = "estimated"
                        
                    except Exception as e:
                        print(f"  ⚠ Auto-estimation failed: {e}")
                        print(f"  → Please specify charge manually for {res_name}")
                        print(f"     Add to KNOWN_CHARGES dict: '{res_name}': <charge>")
                        lig_result["charge_source"] = "failed"
                
                ligand_results.append(lig_result)
                        
            except Exception as e:
                print(f"  ✗ Error: {e}")
                import traceback
                traceback.print_exc()
        else:
            print(f"  ✗ File not found: {lig_file}")
    
    # Summary
    print(f"\n{'='*60}")
    print("SUMMARY")
    print(f"{'='*60}")
    for lr in ligand_results:
        status = "✓" if lr["charge"] is not None else "✗"
        source = f"({lr['charge_source']})" if lr["charge_source"] else ""
        print(f"  {status} {lr['residue']}: charge={lr['charge']} {source}")
else:
    print("No ligand files available. Run previous cell first.")


## Test 6: Antechamber Force Field Generation

Run antechamber to generate GAFF2 parameters with AM1-BCC charges, then validate the frcmod output.


In [ ]:
# Test antechamber force field generation
run_antechamber_robust = get_callable(amber_module.run_antechamber_robust)
validate_frcmod = get_callable(amber_module.validate_frcmod)

# Check if we have ligand results from previous cell
if 'ligand_results' in dir() and ligand_results:
    print("Testing Antechamber Force Field Generation")
    print("=" * 60)
    
    # Store antechamber results
    antechamber_results = []
    
    for lr in ligand_results:
        res_name = lr['residue']
        h_file = lr.get('h_file')
        charge = lr.get('charge')
        
        print(f"\n{'='*60}")
        print(f"Processing: {res_name}")
        print(f"{'='*60}")
        
        if h_file is None:
            print(f"  ✗ No hydrogenated file available")
            continue
            
        if charge is None:
            print(f"  ✗ No charge specified - skipping")
            print(f"     Add '{res_name}' to KNOWN_CHARGES dict with appropriate charge")
            continue
        
        print(f"  Input: {Path(h_file).name}")
        print(f"  Net charge: {charge}")
        
        try:
            # Step 1: Run antechamber
            print(f"\n[Step 1] Running antechamber (GAFF2 + AM1-BCC)...")
            print(f"  This may take a few minutes for large molecules...")
            
            ac_result = run_antechamber_robust(
                ligand_file=h_file,
                net_charge=charge,
                residue_name=res_name[:3].upper(),  # 3-letter code
                charge_method="bcc",
                atom_type="gaff2"
            )
            
            print(f"  ✓ MOL2 output: {Path(ac_result['mol2']).name}")
            print(f"  ✓ FRCMOD output: {Path(ac_result['frcmod']).name}")
            print(f"  ✓ Charge used: {ac_result['charge_used']}")
            print(f"  ✓ Total charge (sum): {ac_result['total_charge']:.4f}")
            
            # Show sqm diagnostics if available
            if ac_result.get('sqm_diagnostics'):
                diag = ac_result['sqm_diagnostics']
                if diag.get('success'):
                    print(f"  ✓ sqm calculation successful")
                else:
                    print(f"  ⚠ sqm issues: {diag.get('errors', [])}")
            
            # Step 2: Validate frcmod
            print(f"\n[Step 2] Validating frcmod...")
            frcmod_result = validate_frcmod(ac_result['frcmod'])
            
            if frcmod_result['valid']:
                print(f"  ✓ frcmod validation PASSED")
            else:
                print(f"  ⚠ frcmod validation WARNINGS:")
                print(f"     ATTN count: {frcmod_result['attn_count']}")
                for w in frcmod_result['warnings'][:3]:
                    print(f"     - {w}")
                if frcmod_result.get('recommendations'):
                    print(f"     Recommendations:")
                    for r in frcmod_result['recommendations'][:2]:
                        print(f"       • {r}")
            
            antechamber_results.append({
                "residue": res_name,
                "mol2": ac_result['mol2'],
                "frcmod": ac_result['frcmod'],
                "charge": ac_result['charge_used'],
                "frcmod_valid": frcmod_result['valid'],
                "success": True
            })
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
            import traceback
            traceback.print_exc()
            antechamber_results.append({
                "residue": res_name,
                "success": False,
                "error": str(e)
            })
    
    # Summary
    print(f"\n{'='*60}")
    print("ANTECHAMBER SUMMARY")
    print(f"{'='*60}")
    for ar in antechamber_results:
        if ar.get('success'):
            frcmod_status = "✓" if ar.get('frcmod_valid') else "⚠"
            print(f"  ✓ {ar['residue']}: charge={ar['charge']}, frcmod={frcmod_status}")
        else:
            print(f"  ✗ {ar['residue']}: FAILED - {ar.get('error', 'unknown')}")
else:
    print("No ligand results available. Run previous cells first.")


## Test 7: tleap System Building

Build the complete MD system with tleap (protein + ligand + solvent + ions).


In [ ]:
# Test tleap system building with ALL ligands
# Reload module to get new build_multi_ligand_system function
import importlib
amber_module = importlib.reload(amber_module)

build_multi_ligand_system = get_callable(amber_module.build_multi_ligand_system)
prepare_protein_for_amber = get_callable(amber_module.prepare_protein_for_amber)

# Check if we have the required files
if 'result' in dir() and 'antechamber_results' in dir():
    print("Testing tleap System Building (ALL LIGANDS)")
    print("=" * 60)
    
    job_dir = Path(result['output_dir'])
    protein_pdb = result.get('protein_pdb')
    
    # Get ALL successful ligand results
    # Each ligand now has a unique filename (e.g., SAH_chainC_gaff.mol2)
    successful_ligands = []
    
    for ar in antechamber_results:
        if ar.get('success') and ar.get('mol2'):
            # Extract residue name from the mol2 filename
            # e.g., ligand_SAH_chainC_H.gaff.mol2 -> SAH
            mol2_name = Path(ar['mol2']).stem  # e.g., ligand_SAH_chainC_H.gaff
            # Remove .gaff suffix if present
            if mol2_name.endswith('.gaff'):
                mol2_name = mol2_name[:-5]
            # Extract residue type (e.g., SAH or LIG1)
            parts = mol2_name.split('_')
            if len(parts) >= 2 and parts[0] == 'ligand':
                resname = parts[1][:3].upper()  # SAH, LIG
            else:
                resname = ar['residue'][:3].upper()
            
            successful_ligands.append({
                'mol2': ar['mol2'],
                'frcmod': ar['frcmod'],
                'residue_name': resname
            })
    
    if protein_pdb is None:
        print("  ✗ No protein PDB available")
    elif not successful_ligands:
        print("  ✗ No successful ligand parameterization available")
    else:
        print(f"  Protein: {Path(protein_pdb).name}")
        print(f"  Ligands: {len(successful_ligands)}")
        for i, lig in enumerate(successful_ligands):
            print(f"    [{i+1}] {lig['residue_name']}: {Path(lig['mol2']).name}")
        
        try:
            # Step 1: Prepare protein with pdb4amber
            print(f"\n[Step 1] Preparing protein with pdb4amber...")
            protein_result = prepare_protein_for_amber(
                protein_pdb,
                output_dir=str(job_dir)
            )
            print(f"  ✓ Prepared protein: {Path(protein_result['output_pdb']).name}")
            if protein_result.get('disulfide_bonds'):
                print(f"  ✓ Disulfide bonds detected: {len(protein_result['disulfide_bonds'])}")
            
            # Step 2: Build system with tleap (ALL LIGANDS)
            print(f"\n[Step 2] Building MD system with tleap...")
            print(f"  Including {len(successful_ligands)} ligands")
            print(f"  Water model: TIP3P")
            print(f"  Box padding: 10.0 Å")
            print(f"  Neutralizing with ions...")
            
            system_result = build_multi_ligand_system(
                protein_pdb=protein_result['output_pdb'],
                ligands=successful_ligands,
                output_dir=str(job_dir),
                forcefield="leaprc.protein.ff14SB",
                water_model="tip3p",
                box_padding=10.0,
                box_type="box",  # cubic box
                neutralize=True,
                salt_conc=0.0
            )
            
            print(f"\n  ✓ System built successfully!")
            print(f"  ✓ Topology file: {Path(system_result['parm7']).name}")
            print(f"  ✓ Coordinate file: {Path(system_result['rst7']).name}")
            print(f"  ✓ Complex PDB: {Path(system_result['complex_pdb']).name}")
            print(f"  ✓ Ligands included: {system_result.get('num_ligands', len(successful_ligands))}")
            print(f"    Names: {system_result.get('ligand_names', [l['residue_name'] for l in successful_ligands])}")
            
            if system_result.get('num_atoms'):
                print(f"  ✓ Total atoms: {system_result['num_atoms']}")
            if system_result.get('num_residues'):
                print(f"  ✓ Total residues: {system_result['num_residues']}")
            
            # Check for warnings
            if system_result.get('warnings'):
                print(f"\n  ⚠ tleap warnings ({len(system_result['warnings'])}):")
                for w in system_result['warnings'][:5]:
                    print(f"     - {w[:80]}...")
            
            # Verify output files exist
            print(f"\n[Step 3] Verifying output files...")
            parm7_path = Path(system_result['parm7'])
            rst7_path = Path(system_result['rst7'])
            
            if parm7_path.exists():
                parm7_size = parm7_path.stat().st_size / 1024  # KB
                print(f"  ✓ {parm7_path.name}: {parm7_size:.1f} KB")
            else:
                print(f"  ✗ {parm7_path.name}: NOT FOUND")
            
            if rst7_path.exists():
                rst7_size = rst7_path.stat().st_size / 1024  # KB
                print(f"  ✓ {rst7_path.name}: {rst7_size:.1f} KB")
            else:
                print(f"  ✗ {rst7_path.name}: NOT FOUND")
            
            # Summary
            print(f"\n{'='*60}")
            print("SYSTEM BUILDING COMPLETE")
            print(f"{'='*60}")
            print(f"  Output directory: {system_result['output_dir']}")
            print(f"  Topology: {system_result['parm7']}")
            print(f"  Coordinates: {system_result['rst7']}")
            print(f"  Ligands: {system_result.get('ligand_names', [])}")
            print(f"\n  These files are ready for MD simulation with Amber/OpenMM!")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
            import traceback
            traceback.print_exc()
else:
    print("Required data not available. Run previous cells first.")


## Test 8: 3D Visualization with py3Dmol

Visualize the built complex with py3Dmol.


In [ ]:
# Visualize tleap build result: Convert parm7/rst7 to PDB and display
import tempfile

try:
    import mdtraj as md
except ImportError:
    print("Installing MDTraj...")
    %pip install mdtraj
    import mdtraj as md

try:
    import py3Dmol
except ImportError:
    print("Installing py3Dmol...")
    %pip install py3Dmol
    import py3Dmol

# Check if we have tleap build results
if 'system_result' in dir() and system_result.get('parm7') and system_result.get('rst7'):
    parm7_path = Path(system_result['parm7'])
    rst7_path = Path(system_result['rst7'])
    
    print("=== tleap Build Validation ===")
    print(f"Topology: {parm7_path.name}")
    print(f"Coordinates: {rst7_path.name}")
    
    if parm7_path.exists() and rst7_path.exists():
        # Load coordinates with MDTraj
        print("\nLoading system with MDTraj...")
        traj = md.load(str(rst7_path), top=str(parm7_path))
        
        print(f"  Total atoms: {traj.n_atoms}")
        print(f"  Total residues: {traj.n_residues}")
        
        # Count residue types
        res_counts = {}
        ligand_resnames = set()
        standard_res = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                        'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                        'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'}
        water_res = {'WAT', 'HOH'}
        ion_res = {'NA', 'CL', 'Na+', 'Cl-', 'K', 'K+'}
        
        for res in traj.topology.residues:
            res_counts[res.name] = res_counts.get(res.name, 0) + 1
            if res.name not in standard_res and res.name not in water_res and res.name not in ion_res:
                ligand_resnames.add(res.name)
        
        # Print summary
        print(f"\n  Residue summary:")
        protein_count = sum(res_counts.get(aa, 0) for aa in standard_res)
        water_count = sum(res_counts.get(w, 0) for w in water_res)
        ion_count = sum(res_counts.get(i, 0) for i in ion_res)
        print(f"    Protein residues: {protein_count}")
        print(f"    Water molecules: {water_count}")
        print(f"    Ions: {ion_count}")
        print(f"    Ligands: {ligand_resnames}")
        for lig in ligand_resnames:
            print(f"      - {lig}: {res_counts.get(lig, 0)}")
        
        # Convert to PDB for visualization
        print("\nConverting to PDB for visualization...")
        with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
            tmp_pdb = tmp.name
        traj.save_pdb(tmp_pdb)
        
        with open(tmp_pdb, 'r') as f:
            pdb_content = f.read()
        Path(tmp_pdb).unlink()
        
        # Create viewer
        print("Creating 3D view...")
        view = py3Dmol.view(width=900, height=700)
        view.addModel(pdb_content, 'pdb')
        
        # Style protein - cartoon
        view.setStyle({'resn': list(standard_res)}, 
                      {'cartoon': {'color': 'spectrum'}})
        
        # Style ligands - stick with different colors
        lig_colors = ['green', 'cyan', 'magenta', 'orange']
        for i, resn in enumerate(sorted(ligand_resnames)):
            color = lig_colors[i % len(lig_colors)]
            view.setStyle({'resn': resn}, 
                          {'stick': {'color': color, 'radius': 0.3}})
            # Add label
            view.addResLabels({'resn': resn}, {
                'fontSize': 12,
                'fontColor': 'white',
                'backgroundColor': color,
                'backgroundOpacity': 0.8
            })
        
        # Style water - small dots (blue)
        view.setStyle({'resn': list(water_res)}, 
                      {'sphere': {'radius': 0.15, 'color': 'lightblue'}})
        
        # Style ions - spheres
        view.setStyle({'resn': ['NA', 'Na+']}, 
                      {'sphere': {'radius': 0.8, 'color': 'purple'}})
        view.setStyle({'resn': ['CL', 'Cl-']}, 
                      {'sphere': {'radius': 0.8, 'color': 'yellow'}})
        
        view.zoomTo()
        
        # Use orthographic projection
        view.setProjection('orthographic')
        
        # Add box visualization if available
        if traj.unitcell_lengths is not None:
            box = traj.unitcell_lengths[0] * 10  # nm to Angstrom
            print(f"\n  Box dimensions: {box[0]:.1f} x {box[1]:.1f} x {box[2]:.1f} Å")
        
        print(f"\n🔹 Protein ({protein_count} res): Cartoon (spectrum)")
        print(f"🔹 Ligands {list(ligand_resnames)}: Stick (colored)")
        print(f"🔹 Water ({water_count} mol): Dots (light blue)")
        print(f"🔹 Ions ({ion_count}): Spheres (Na+=purple, Cl-=yellow)")
        
        view.show()
    else:
        print(f"Files not found:")
        print(f"  parm7: {parm7_path.exists()}")
        print(f"  rst7: {rst7_path.exists()}")
else:
    print("No tleap build results available. Run Test 7 first.")


## Test 9: OpenMM Simulation (Minimize → Equilibrate → Production)

Run a minimal MD simulation with OpenMM to verify the system works.
- Platform: CUDA > OpenCL (Mac GPU) > CPU
- Ensemble: NPT (1 atm, 300 K)
- Short run for testing


In [ ]:
# OpenMM Simulation: Minimize → Equilibrate → Production
import time

try:
    import openmm as mm
    from openmm import app, unit
    from openmm.app import PDBFile, AmberPrmtopFile, AmberInpcrdFile
    from openmm.app import Simulation, StateDataReporter, DCDReporter
except ImportError:
    print("Installing OpenMM...")
    %pip install openmm
    import openmm as mm
    from openmm import app, unit
    from openmm.app import PDBFile, AmberPrmtopFile, AmberInpcrdFile
    from openmm.app import Simulation, StateDataReporter, DCDReporter

def select_platform():
    """Select best available platform: CUDA > OpenCL > CPU"""
    platform_preference = ['CUDA', 'OpenCL', 'CPU']
    
    print("Checking available platforms...")
    for name in platform_preference:
        try:
            platform = mm.Platform.getPlatformByName(name)
            # Test if platform actually works
            if name == 'CUDA':
                try:
                    platform.getPropertyDefaultValue('DeviceIndex')
                    print(f"  ✓ {name} available")
                    return platform, name
                except Exception:
                    print(f"  ✗ {name} not available (no GPU)")
                    continue
            elif name == 'OpenCL':
                print(f"  ✓ {name} available (Mac GPU)")
                return platform, name
            else:
                print(f"  ✓ {name} available")
                return platform, name
        except Exception as e:
            print(f"  ✗ {name} not available: {e}")
    
    raise RuntimeError("No suitable platform found!")

# Check if we have the topology/coordinate files
if 'system_result' in dir() and system_result.get('parm7') and system_result.get('rst7'):
    parm7_path = system_result['parm7']
    rst7_path = system_result['rst7']
    output_dir = Path(system_result['output_dir'])
    
    print("=" * 60)
    print("OpenMM MD Simulation")
    print("=" * 60)
    print(f"Topology: {Path(parm7_path).name}")
    print(f"Coordinates: {Path(rst7_path).name}")
    
    # Select platform
    platform, platform_name = select_platform()
    print(f"\n→ Using platform: {platform_name}")
    
    # Simulation parameters
    temperature = 300 * unit.kelvin
    pressure = 1 * unit.atmosphere
    timestep = 2 * unit.femtoseconds
    friction = 1 / unit.picosecond
    
    # Short runs for testing
    minimize_max_iter = 500
    equil_steps = 2500      # 5 ps equilibration
    prod_steps = 5000       # 10 ps production
    report_interval = 500   # Report every 1 ps
    
    print(f"\nSimulation parameters:")
    print(f"  Temperature: {temperature}")
    print(f"  Pressure: {pressure}")
    print(f"  Timestep: {timestep}")
    print(f"  Equilibration: {equil_steps} steps ({equil_steps * 2 / 1000} ps)")
    print(f"  Production: {prod_steps} steps ({prod_steps * 2 / 1000} ps)")
    
    # Load system
    print(f"\n[Step 1] Loading system...")
    t0 = time.time()
    prmtop = AmberPrmtopFile(parm7_path)
    inpcrd = AmberInpcrdFile(rst7_path)
    print(f"  ✓ Loaded in {time.time() - t0:.1f}s")
    print(f"  Atoms: {prmtop.topology.getNumAtoms()}")
    
    # Create system
    print(f"\n[Step 2] Creating OpenMM system...")
    t0 = time.time()
    system = prmtop.createSystem(
        nonbondedMethod=app.PME,
        nonbondedCutoff=10 * unit.angstrom,
        constraints=app.HBonds,
        rigidWater=True
    )
    
    # Add barostat for NPT
    system.addForce(mm.MonteCarloBarostat(pressure, temperature, 25))
    print(f"  ✓ System created in {time.time() - t0:.1f}s")
    
    # Create integrator and simulation
    integrator = mm.LangevinMiddleIntegrator(temperature, friction, timestep)
    simulation = Simulation(prmtop.topology, system, integrator, platform)
    simulation.context.setPositions(inpcrd.positions)
    
    if inpcrd.boxVectors is not None:
        simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
    
    # Energy minimization
    print(f"\n[Step 3] Energy minimization (max {minimize_max_iter} steps)...")
    t0 = time.time()
    state_before = simulation.context.getState(getEnergy=True)
    energy_before = state_before.getPotentialEnergy().value_in_unit(unit.kilojoules_per_mole)
    
    simulation.minimizeEnergy(maxIterations=minimize_max_iter)
    
    state_after = simulation.context.getState(getEnergy=True)
    energy_after = state_after.getPotentialEnergy().value_in_unit(unit.kilojoules_per_mole)
    print(f"  ✓ Minimized in {time.time() - t0:.1f}s")
    print(f"  Energy: {energy_before:.1f} → {energy_after:.1f} kJ/mol")
    
    # Initialize velocities
    simulation.context.setVelocitiesToTemperature(temperature)
    
    # Setup reporters
    dcd_file = output_dir / "trajectory.dcd"
    log_file = output_dir / "simulation.log"
    
    simulation.reporters.append(DCDReporter(str(dcd_file), report_interval))
    simulation.reporters.append(StateDataReporter(
        str(log_file), report_interval,
        step=True, time=True, potentialEnergy=True, kineticEnergy=True,
        totalEnergy=True, temperature=True, volume=True, density=True,
        speed=True
    ))
    simulation.reporters.append(StateDataReporter(
        sys.stdout, report_interval,
        step=True, time=True, temperature=True, speed=True, remainingTime=True,
        totalSteps=equil_steps + prod_steps
    ))
    
    # Equilibration (NVT heating is implicit, we go straight to NPT)
    print(f"\n[Step 4] NPT Equilibration ({equil_steps * 2 / 1000} ps)...")
    t0 = time.time()
    simulation.step(equil_steps)
    print(f"  ✓ Equilibration done in {time.time() - t0:.1f}s")
    
    # Production
    print(f"\n[Step 5] Production ({prod_steps * 2 / 1000} ps)...")
    t0 = time.time()
    simulation.step(prod_steps)
    print(f"  ✓ Production done in {time.time() - t0:.1f}s")
    
    # Save final state
    final_pdb = output_dir / "final_state.pdb"
    state = simulation.context.getState(getPositions=True, getVelocities=True)
    with open(final_pdb, 'w') as f:
        PDBFile.writeFile(simulation.topology, state.getPositions(), f)
    print(f"\n✓ Final state saved: {final_pdb.name}")
    
    # Summary
    print(f"\n{'='*60}")
    print("SIMULATION COMPLETE")
    print(f"{'='*60}")
    print(f"  Output directory: {output_dir}")
    print(f"  Trajectory: {dcd_file.name}")
    print(f"  Log: {log_file.name}")
    print(f"  Final PDB: {final_pdb.name}")
    
else:
    print("No topology/coordinate files available. Run system building first.")


## Test 10: Trajectory Visualization with py3Dmol

Visualize the MD trajectory (equilibration + production) with py3Dmol animation.


In [ ]:
# Trajectory visualization with py3Dmol
# Note: Only LIG1 is in the MD system (SAH was not included in tleap build)
import numpy as np
import tempfile

try:
    import mdtraj as md
except ImportError:
    print("Installing MDTraj...")
    %pip install mdtraj
    import mdtraj as md

import py3Dmol

# Check if we have trajectory files
if 'system_result' in dir() and system_result.get('output_dir'):
    output_dir = Path(system_result['output_dir'])
    dcd_file = output_dir / "trajectory.dcd"
    parm7_file = Path(system_result['parm7'])
    
    if dcd_file.exists() and parm7_file.exists():
        print("Loading trajectory...")
        
        # Load trajectory with MDTraj
        traj = md.load(str(dcd_file), top=str(parm7_file))
        print(f"  Frames: {traj.n_frames}")
        print(f"  Atoms: {traj.n_atoms}")
        print(f"  Time: {traj.time[0]:.1f} - {traj.time[-1]:.1f} ps")
        
        # Select protein and ligand atoms (exclude water and ions)
        protein_indices = traj.topology.select('protein')
        
        # Find ALL ligand residues (LIG, SAH, or any non-standard non-water residue)
        lig_indices = []
        ligand_resnames = set()
        ligand_details = []  # For debugging
        standard_res = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                        'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                        'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL',
                        'WAT', 'HOH', 'NA', 'CL', 'Na+', 'Cl-'}
        for residue in traj.topology.residues:
            if residue.name not in standard_res:
                atom_indices = [atom.index for atom in residue.atoms]
                lig_indices.extend(atom_indices)
                ligand_resnames.add(residue.name)
                ligand_details.append(f"{residue.name}:{residue.resSeq} ({len(atom_indices)} atoms)")
        lig_indices = np.array(lig_indices) if lig_indices else np.array([], dtype=int)
        
        # Debug: show all found ligands
        print(f"  Found ligand residues:")
        for detail in ligand_details[:10]:  # Limit output
            print(f"    - {detail}")
        if len(ligand_details) > 10:
            print(f"    ... and {len(ligand_details) - 10} more")
        
        # Combine protein + ligand
        if len(lig_indices) > 0:
            keep_indices = np.concatenate([protein_indices, lig_indices])
        else:
            keep_indices = protein_indices
        
        # Remove duplicates and sort
        keep_indices = np.unique(keep_indices)
        
        # Ensure indices are within range
        keep_indices = keep_indices[keep_indices < traj.n_atoms]
        
        print(f"  Protein atoms: {len(protein_indices)}")
        print(f"  Ligand atoms: {len(lig_indices)}")
        print(f"  Ligand types: {ligand_resnames if ligand_resnames else 'None'}")
        print(f"  Selection atoms: {len(keep_indices)}")
        
        # Subset trajectory to protein + ligand only
        traj_subset = traj.atom_slice(keep_indices)
        print(f"  Visualization atoms: {traj_subset.n_atoms}")
        
        # Sample frames for visualization
        max_frames = 15
        if traj_subset.n_frames > max_frames:
            frame_indices = np.linspace(0, traj_subset.n_frames - 1, max_frames, dtype=int)
            traj_viz = traj_subset[frame_indices]
            print(f"  Sampled {max_frames} frames for visualization")
        else:
            traj_viz = traj_subset
        
        print("\nPreparing visualization...")
        
        # Save all frames to a single multi-model PDB file
        # This is the proper way to do trajectory animation in py3Dmol
        with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
            tmp_path = tmp.name
        
        # Write all frames as MODEL/ENDMDL blocks
        with open(tmp_path, 'w') as f:
            for frame_idx in range(traj_viz.n_frames):
                frame = traj_viz[frame_idx]
                # Save single frame to temp
                frame_tmp = tmp_path + f".frame{frame_idx}.pdb"
                frame.save_pdb(frame_tmp, force_overwrite=True)
                with open(frame_tmp, 'r') as ff:
                    content = ff.read()
                # Wrap in MODEL/ENDMDL
                f.write(f"MODEL     {frame_idx + 1}\n")
                # Remove any existing MODEL/ENDMDL lines
                for line in content.split('\n'):
                    if not line.startswith('MODEL') and not line.startswith('ENDMDL') and line.strip():
                        f.write(line + '\n')
                f.write("ENDMDL\n")
                Path(frame_tmp).unlink()
        
        # Read the multi-model PDB
        with open(tmp_path, 'r') as f:
            pdb_content = f.read()
        Path(tmp_path).unlink()
        
        # Create viewer
        view = py3Dmol.view(width=800, height=600)
        view.addModelsAsFrames(pdb_content, 'pdb')
        
        # Style - apply to all frames
        aa_list = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                   'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                   'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
        
        view.setStyle({'resn': aa_list}, {'cartoon': {'color': 'spectrum'}})
        
        # Style all ligands with different colors and add labels
        lig_colors = ['green', 'cyan', 'magenta', 'orange']
        for i, resn in enumerate(sorted(ligand_resnames)):
            color = lig_colors[i % len(lig_colors)]
            view.setStyle({'resn': resn}, {'stick': {'color': color, 'radius': 0.3}})
            # Add label for each ligand (at center of residue)
            view.addResLabels({'resn': resn}, {
                'fontSize': 12,
                'fontColor': 'white',
                'backgroundColor': color,
                'backgroundOpacity': 0.8
            })
        
        view.zoomTo()
        
        # Use orthographic projection
        view.setProjection('orthographic')
        
        # Enable frame-based animation
        view.animate({'loop': 'forward', 'reps': 0, 'interval': 100})
        
        print(f"\n🔹 Protein: Cartoon (spectrum)")
        print(f"🔹 Ligands: {list(ligand_resnames)} (stick, different colors)")
        print(f"🔹 Frames: {traj_viz.n_frames} (animated)")
        print(f"\n▶️ Animation should auto-play")
        print(f"   If static, try opening in browser or use nglview instead")
        
        view.show()
    else:
        print(f"Trajectory file not found: {dcd_file}")
else:
    print("No simulation results available. Run simulation first.")
